# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'internet-firewall-automl'
project_folder = './pipeline-project-automl'

experiment=Experiment(ws, experiment_name)
experiment

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ED2BSHUV2 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


Name,Workspace,Report Page,Docs Page
internet-firewall-automl,quick-starts-ws-138160,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cc-sta-ds12-2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Creating
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


**Dataset**

**Internet Firewall Data Data Set**

**Data Set Information:**
There are 12 features in total. Action feature is used as a class. There are 4 classes in total. These are allow, action, drop and reset-both classes.


**Attribute Information:**
Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received

Link: https://archive.ics.uci.edu/ml/datasets/Internet+Firewall+Data

In [4]:
# NOTE: update the key to match the dataset name
found = False
key = "internet-firewall-dataset"
description_text = "Internet Firewall Data Data Set"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00542/log2.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [5]:
df.describe()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
count,65532.000000,65532.000000,65532.000000,65532.000000,6.553200e+04,6.553200e+04,6.553200e+04,6.553200e+04,65532.000000,65532.000000,65532.000000
mean,49391.969343,10577.385812,19282.972761,2671.049930,9.712395e+04,2.238580e+04,7.473815e+04,1.028660e+02,65.833577,41.399530,61.466505
std,15255.712537,18466.027039,21970.689669,9739.162278,5.618439e+06,3.828139e+06,2.463208e+06,5.133002e+03,302.461762,3218.871288,2223.332271
min,0.000000,0.000000,0.000000,0.000000,6.000000e+01,6.000000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
25%,49183.000000,80.000000,0.000000,0.000000,6.600000e+01,6.600000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
50%,53776.500000,445.000000,8820.500000,53.000000,1.680000e+02,9.000000e+01,7.900000e+01,2.000000e+00,15.000000,1.000000,1.000000
75%,58638.000000,15000.000000,38366.250000,443.000000,7.522500e+02,2.100000e+02,4.490000e+02,6.000000e+00,30.000000,3.000000,2.000000
max,65534.000000,65535.000000,65535.000000,65535.000000,1.269359e+09,9.484772e+08,3.208818e+08,1.036116e+06,10824.000000,747520.000000,327208.000000


In [6]:
df.groupby(['Action']).count()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
Action,,,,,,,,,,,
allow,37640,37640,37640,37640,37640,37640,37640,37640,37640,37640,37640
deny,14987,14987,14987,14987,14987,14987,14987,14987,14987,14987,14987
drop,12851,12851,12851,12851,12851,12851,12851,12851,12851,12851,12851
reset-both,54,54,54,54,54,54,54,54,54,54,54


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    'model_explainability': True
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Action",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails

RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Train-Test data split
STATUS:       DONE
DESCRIPTION:  Your input data has been split into a training dataset and a holdout test dataset for validation of the model. The test holdout dataset reflects the original distribution of your input data.
              
DETAILS:      
+---------------------------------+---------------------------------+---------------------------------+
|Dataset                          |Row counts                       |Percentage                       |
+=================================+=================================+=================================+
|train                            |58978                            |89.99

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
# Retrieve and save your best automl model.
automl_best_run, automl_fitted_model = automl_run.get_output()


# Reference: https://knowledge.udacity.com/questions/357775 
print("Fitted model: ", automl_fitted_model.steps[-1])
print("---"*5)
print("AutoML best run accuracy: ", automl_best_run.get_metrics(name="accuracy"))
print("---"*5)
print("AutoML run Summary: ", automl_run.summary())


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Fitted model:  ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('25',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f96a8f21cf8>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                      

In [11]:
automl_estimator = automl_fitted_model.steps[-1]
print(automl_estimator)

('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('25',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f96a8f21cf8>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsamp

In [12]:
automl_best_run

Experiment,Id,Type,Status,Details Page,Docs Page
internet-firewall-automl,AutoML_b5822113-79c0-4b5d-b65d-8edec41201e4_38,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
# model.get_file_names()


In [14]:
import joblib
import os

try:
    os.mkdir('outputs')
except:
    print('outputs already exists')

# automl_best_run.register_model(model_name = 'automl_best_model.pkl', model_path = './outputs/')
joblib.dump(automl_fitted_model, "./outputs/automl_fitted_model.pkl")

model_name = 'Internet-firewall-AML-model'

model = automl_best_run.register_model(model_name = model_name, model_path = 'outputs/model.pkl')
print(model.name, model.id, model.version, sep='\t')

Internet-firewall-AML-model	Internet-firewall-AML-model:1	1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [26]:
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig, Model
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice, Webservice

# Register the model to deploy
# model = automl_best_run.register_model(model_name = "mymodel", model_path = "outputs/model.pkl")
automl_best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

env = Environment.from_conda_specification('myenv', 'myenv.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True,tags = {'name':'Action Prediction'},description='Internet Firewall Data')
service = model.deploy(workspace=ws,name="internet-firewall-automl-deploy",models=[model],inference_config=inference_config,deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................

In [ ]:
print(service.get_logs())

In [ ]:
service.update(enable_app_insights=True)

print(service.state)
print("scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
import json
import requests
# URL for the web service
scoring_uri = 'http://d36e6c91-ade6-4343-bbe9-cf0f5f29288f.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'F0rlg2IVfHY4WmoPx5kLxjQxA3iWwWSH'

data = {
  "data": [
    {#ALLOW
                  'Source Port': 50148, 
                  'Destination Port': 80, 
                  'NAT Source Port': 57004,
                  'NAT Destination Port': 80,
                  'Bytes': 9058,
                  'Bytes Sent': 868,
                  'Bytes Received': 8190,
                  'Packets': 18,
                  'Elapsed Time (sec)': 46,
                  'pkts_sent': 10,
                  'pkts_received': 8
                 
                 
    },
    {#DENY
                  'Source Port': 56103, 
                  'Destination Port': 21247, 
                  'NAT Source Port': 0,
                  'NAT Destination Port': 0,
                  'Bytes': 66,
                  'Bytes Sent': 66,
                  'Bytes Received': 0,
                  'Packets': 1,
                  'Elapsed Time (sec)': 0,
                  'pkts_sent': 1,
                  'pkts_received': 0
                 
                 
    },
    {#DROP
                  'Source Port': 49783, 
                  'Destination Port': 445, 
                  'NAT Source Port': 0,
                  'NAT Destination Port': 0,
                  'Bytes': 70,
                  'Bytes Sent': 70,
                  'Bytes Received': 0,
                  'Packets': 1,
                  'Elapsed Time (sec)': 0,
                  'pkts_sent': 1,
                  'pkts_received': 0
                 
                 
    }
 ]
}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)
#print(f"Predicted: {output}\nExpected: {y}")

headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print("prediction ", resp.text)

prediction  ["allow", "deny", "drop"]


TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
#print the logs of the web service
web_service_logs = service.get_logs()
with open("./web_service_logs.txt", 'w') as output:
    output.write(str(web_service_logs))

print(web_service_logs)
#delete the service
service.delete()


2021-02-08T16:21:54,305656700+00:00 - gunicorn/run 
2021-02-08T16:21:54,307644500+00:00 - rsyslog/run 
2021-02-08T16:21:54,333748800+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libuuid.so.1: no version information available (required by rsyslogd)
2021-02-08T16:21:54,386133100+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [30]:
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

